In [2]:
import pandas as pd
import pygsheets

import os
from datetime import datetime, timedelta
import pickle
import json

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [10]:
    
SHEET_NAME = "HEROKU"

# if DEBUG is True:
#     SHEET_NAME = "TEST | STUDY-PLAN"

SPREADSHEET_ID = "1MQOyYpuJ4zF5xyqjVtAnrTrG3In9lxWl9L3yXiTeYiw"

# Get real path of executable directory
# dir_name = os.path.dirname(os.path.realpath(sys.argv[0]))

secretpath = "creds_google/service_account.json"

gc = pygsheets.authorize(service_file=secretpath)

workbook = gc.open_by_key(SPREADSHEET_ID)

worksheet = workbook.worksheet_by_title(SHEET_NAME)

# Retrieve the data from the sheets
# df = worksheet.get_as_df(start="A", end="I")

# Have an empty dataframe to write back to the sheets

# new_df = pd.DataFrame(columns=df.columns)
# new_df.head()


In [30]:
# worksheet.set_dataframe(df, start='A2', copy_head=False)
import requests
import urllib

URI = 'https://gis-api.aiesec.org/graphql'
ACCESS_TOKEN = "7789a122c78d710a724f3cd2a7d8daf809b467d46cf63eafa76b2ae87a32f035"
ORIGINAL_QUERY = '{allOpportunityApplication(filters: {created_at: {from:"2021-01-01", to:"2021-03-20"}}){data{id status created_at date_matched date_approved date_realized experience_start_date experience_end_date date_approval_broken nps_response_completed_at updated_at person{id full_name home_mc{id name}home_lc{id name}}host_lc{id name}home_mc{id name}opportunity{id created_at title available_openings duration sub_product{name}programme{id short_name_display}}standards{option}}}}'
QUERY = '{allOpportunityApplication(filters: {created_at: {from:"2021-01-01", to:"2021-03-20"}}){data{id status created_at date_matched date_approved date_realized experience_start_date experience_end_date date_approval_broken nps_response_completed_at updated_at person{id full_name home_mc{name}home_lc{name}}host_lc{name}home_mc{name}opportunity{id created_at title duration sub_product{name}programme{short_name_display}}standards{option}}}}'



url = f"{URI}?query={QUERY}&access_token={ACCESS_TOKEN}"


url = urllib.parse.quote(url, safe='~@#$&()*!+=:;,.?/\'')


response = requests.post(url, data={ 'Content-Type': 'application/json'})


In [32]:

results = json.loads(response.content)

# Reduce the dict by 3
results = results['data']['allOpportunityApplication']['data']

#  Flatten dictionary and compress keys
results = pd.json_normalize(results, sep='_')
import numpy as np
results.replace([np.NaN], '-', inplace=True)
print(results.iloc[0])

id                                                                          6122201
status                                                                         open
created_at                                                     2021-03-18T07:46:42Z
date_matched                                                                      -
date_approved                                                                     -
date_realized                                                                     -
experience_start_date                                                             -
experience_end_date                                                               -
date_approval_broken                                                              -
nps_response_completed_at                                                         -
updated_at                                                     2021-03-18T07:46:42Z
standards                                                                   

In [33]:
worksheet.set_dataframe(results, start='A1', copy_head=True)
# results.info()
# print("Number of Applicants :", len(results.person_id.unique()))
# print("Number of Applications :", results.shape[0])
# print(results.person_id)
print(results.groupby('person_home_lc_name')['person_id'].value_counts())

person_home_lc_name  person_id
(Closed RCSI)        1002581      2
Awal                 4564862      6
                     1725431      1
                     4576705      1
Dilmun               4534283      2
Tylos                2011401      1
                     4531535      1
Name: person_id, dtype: int64


In [18]:
    results[results.person_home_lc_name == 'Awal'][['person_full_name', 'opportunity_programme_short_name_display'] ]
    # results.info()

,person_full_name,opportunity_programme_short_name_display
0,Fatema Alafoo,GTe
1,Fatema Alafoo,GTe
2,Fatema Alafoo,GTe
4,Touqeer Mohammad,GTa
5,Fatema Alafoo,GTa
6,Fatema Alafoo,GTa
7,Fatema Alafoo,GTa
10,Hanan Ali Ateya Jasim,GTa
